In [40]:
# import sounddevice as sd
# import scipy.io.wavfile as wav
# fs = 44100  # Sampling frequency
# duration = 10  # Duration in seconds
# print("Start talking")
# myrecording = sd.rec(duration * fs, samplerate=fs, channels=2, dtype='int16')
# sd.wait()  # Wait until recording is finished
# print("Finished talking")
#
# # Save to file
# wav.write("output.wav", fs, myrecording)

Start talking
Finished talking


In [ ]:
import wave
import pyaudio
#Records an audio and saves it to a file
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 10
WAVE_OUTPUT_FILENAME = "output.wav"
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)
print("* recording")
frames = []
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* recording done")
stream.stop_stream()
stream.close()
p.terminate()
wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

In [1]:
import torch
from transformers import pipeline
import torchaudio
import torchaudio.transforms as T
#Pre-process the audio
waveform, sample_rate = torchaudio.load('output.wav')
if waveform.shape[0] > 1:
   waveform = waveform.mean(dim=0)
resampler = T.Resample(orig_freq=sample_rate, new_freq=16000)
waveform = resampler(waveform)
waveform = waveform.squeeze()
waveform_numpy = waveform.numpy()

In [2]:
# orig_freq = sample_rate
# new_freq = 16000
#
# resampler = T.Resample(orig_freq=orig_freq, new_freq=new_freq)
# waveform = resampler(waveform)

In [3]:
# print(torch.cuda.device_count())
# for i in range(torch.cuda.device_count()):
#     print(f"Device {i}: {torch.cuda.get_device_name(i)}")
#
# print(torch.cuda.current_device())
# print(f"Active GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")
#
# torch.cuda.set_device(1)  # Set to GPU 1 (0-based indexing)
# print(f"Using device: {torch.cuda.get_device_name(torch.cuda.current_device())}")
#torch.cuda.set_device(1)
#print(f"Using device: {torch.cuda.get_device_name(torch.cuda.current_device())}")

2
Device 0: NVIDIA GeForce RTX 4070 SUPER
Device 1: NVIDIA GeForce GTX 1660 Ti
0
Active GPU: NVIDIA GeForce RTX 4070 SUPER
Using device: NVIDIA GeForce GTX 1660 Ti


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = pipeline("automatic-speech-recognition",model="openai/whisper-large-v2",chunk_length_s=30,device=device)
prediction = pipe(waveform_numpy, batch_size=8)
print(prediction["text"])

C:\Python\PyProjects\WhisperVenv\.venv\Lib\site-packages\transformers\models\whisper\generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pa

 Ok, so now I'm testing another way to record this audio and that's it. Let's find out what happens.


C:\Python\PyProjects\WhisperVenv\.venv\Lib\site-packages\transformers\models\whisper\generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.